In [62]:
import lyricwikia
import pandas as pd

df = pd.read_csv("pop_playlist_project.csv")
df.columns = ['Spotify_URI','TrackName','ArtistName','AlbumName','DiscNum','TrackNum','TrackDur','AddedBy','AddedAt']
lyrics = ""

for i in range(len(df)):
    try:
        lyrics_temp = lyricwikia.get_lyrics(df.ArtistName[i], df.TrackName[i])
        lyrics=lyrics+lyrics_temp
    except:
        continue







In [63]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

In [64]:
import re
raw_text = lyrics.lower()
wordList = re.sub('[^\w]',' ',raw_text).split()
word_list =[]
#append to word_list to get rid of any duplicate words
wl = [word_list.append(item) for item in wordList if item not in word_list]
    
    
#chars = sorted(list(set(raw_text)))
word_to_int = dict((c,i) for i, c in enumerate(word_list))



n_words = len(wordList)
n_vocab = len(word_list)
print("Total Words: ", n_words)
print("Total Vocab: ",n_vocab)

Total Words:  162759
Total Vocab:  5081


In [65]:
seq_length = 20
dataX = []
dataY = []



for i in range(0,n_words - seq_length,1):
    seq_in = wordList[i:i + seq_length]
    #seq_in = re.sub('[^\w]',' ',seq_in).split()

    seq_out = wordList[i + seq_length]
    
    dataX.append([word_to_int[string] for string in seq_in])
    
    
    dataY.append(word_to_int[seq_out])
    
n_patterns = len(dataX)
print("Total Patterns: ",n_patterns)


Total Patterns:  162739


In [70]:

X = numpy.reshape(dataX,(n_patterns,seq_length,1))
X = X/float(n_vocab)
y=np_utils.to_categorical(dataY)
print(X)
print(y)



[[[0.        ]
  [0.00019681]
  [0.00039362]
  ...
  [0.00295217]
  [0.00314899]
  [0.0033458 ]]

 [[0.00019681]
  [0.00039362]
  [0.00059043]
  ...
  [0.00314899]
  [0.0033458 ]
  [0.00354261]]

 [[0.00039362]
  [0.00059043]
  [0.00078725]
  ...
  [0.0033458 ]
  [0.00354261]
  [0.00373942]]

 ...

 [[0.03070262]
  [0.03089943]
  [0.03070262]
  ...
  [0.03089943]
  [0.03070262]
  [0.03089943]]

 [[0.03089943]
  [0.03070262]
  [0.03089943]
  ...
  [0.03070262]
  [0.03089943]
  [0.03070262]]

 [[0.03070262]
  [0.03089943]
  [0.043889  ]
  ...
  [0.03089943]
  [0.03070262]
  [0.03089943]]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [67]:
model = Sequential()
model.add(LSTM(256,input_shape=(X.shape[1],X.shape[2]),return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [68]:

model.fit(X,y,epochs=20,batch_size=128)

Epoch 1/20
162739/162739 [==============================] - 786s 5ms/step - loss: 6.0926
Epoch 2/20
162739/162739 [==============================] - 859s 5ms/step - loss: 6.0421
Epoch 3/20
162739/162739 [==============================] - 860s 5ms/step - loss: 6.0445
Epoch 4/20
162739/162739 [==============================] - 1341s 8ms/step - loss: 6.0464
Epoch 5/20
162739/162739 [==============================] - 1392s 9ms/step - loss: 6.0448
Epoch 6/20
162739/162739 [==============================] - 552s 3ms/step - loss: 6.0469
Epoch 7/20
162739/162739 [==============================] - 538s 3ms/step - loss: 6.0501
Epoch 8/20
162739/162739 [==============================] - 539s 3ms/step - loss: 6.0508
Epoch 9/20
162739/162739 [==============================] - 541s 3ms/step - loss: 6.0516
Epoch 10/20
162739/162739 [==============================] - 547s 3ms/step - loss: 6.0522
Epoch 11/20
162739/162739 [==============================] - 548s 3ms/step - loss: 6.0538
Epoch 12/20
16273

In [69]:
int_to_word = dict((i,c) for i, c in enumerate(word_list))

#start = numpy.random.randint(0,len(dataX)-1)
#pattern = dataX[start]
initial_text='this is a song that says me who i am and how to make words he says she says i'
initial_text = re.sub('[^\w]',' ',initial_text).split()
print(initial_text)
pattern=[word_to_int[string] for string in initial_text]
pattern=pattern[0:seq_length]

#initial_text will be our user input, here it must be about 7 chars in length(including spaces)"
text_gen=""

for i in range(100):
    x = numpy.reshape(pattern,(1,len(pattern),1))
    x = x/float(n_vocab)
    prediction = model.predict(x,verbose=0)
    prediction = prediction[0]
    index = numpy.random.choice(len(prediction),p=prediction)
    #index = numpy.argmax(prediction)
    result = int_to_word[index]
    text_gen = text_gen +' ' + result
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print(text_gen)
    


['this', 'is', 'a', 'song', 'that', 'says', 'me', 'who', 'i', 'am', 'and', 'how', 'to', 'make', 'words', 'he', 'says', 'she', 'says', 'i']
 one to baby god she t little cha who she it it for just pullin back late sound won now now up brings slow me never cause and tell just just but red i on and to see and right on happened take keep feel i y it a oh i know this t i waiting by grab mine feel of ever ran stone with that where more i my tears as i just me that all baby where thank and go ain forget my time all but again like too smile yes home you alive my dare re it


In [49]:
model.fit(X,y,epochs=10,batch_size=150)

Epoch 1/10
8503/8503 [==============================] - 7s 781us/step - loss: 2.8042
Epoch 2/10
8503/8503 [==============================] - 7s 792us/step - loss: 2.7705
Epoch 3/10
8503/8503 [==============================] - 7s 771us/step - loss: 2.7209
Epoch 4/10
8503/8503 [==============================] - 6s 759us/step - loss: 2.6837
Epoch 5/10
8503/8503 [==============================] - 7s 816us/step - loss: 2.6466
Epoch 6/10
8503/8503 [==============================] - 7s 778us/step - loss: 2.6120
Epoch 7/10
8503/8503 [==============================] - 7s 840us/step - loss: 2.5862
Epoch 8/10
8503/8503 [==============================] - 7s 775us/step - loss: 2.5545
Epoch 9/10
8503/8503 [==============================] - 7s 783us/step - loss: 2.5326
Epoch 10/10
8503/8503 [==============================] - 7s 810us/step - loss: 2.5006


In [51]:
int_to_char = dict((i, c) for i, c in enumerate(chars))
#initial_text="these lyrics are for my new great rock song about iupui and these lyrics are for my new great rock song about data"
#pattern=[char_to_int[char] for char in initial_text]
#pattern=pattern[0:seq_length]

start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]

text_gen=""

# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    prediction = prediction[0]
#    index = numpy.random.choice(len(prediction), p=prediction)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    text_gen=text_gen+result
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

print(text_gen)

ee to tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou to